<a href="https://colab.research.google.com/github/barreler126/BackgroundRemoval/blob/main/SD_CLSA_webui_Drive_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 挂载Google云盘部署

# 使用说明：


*   代码基于https://github.com/camenduru/ 中web UI内容进行了修改调整
*   由于官方对免费用户限制，修改代码绕开运行警告
*   Pro用户同样可用
*   云盘免费空间为15G,因此下方Controlnet模型只下载了部分
*   若已扩容或有需要，可以打开下方代码中绿色注释部分
*   若一些插件或者模型不需要，可以注释或者删除
*   默认下载了v1-5-pruned-emaonly.ckpt，不需要可以删除







## 使用方法：


首次运行步骤：


1.   新建网页，登录谷歌云盘(https://drive.google.com/)
点击左上角新建—更多-关联更多应用-搜索Colaboratory，点击进入安装
2.   在该文档上方点击【复制到云端硬盘】，或者选择上方【文件】-【在云端硬盘中保存一份副本】，页面会自动跳转至副本窗口，重命名为【SD_CLSA_webui_Drive_colab.ipynb】后关闭本页面
3.   在副本页面中，选择上方【代码执行程序】-【更改运行时类型】，免费用户GPU选择【T4】，Pro用户可选【A100】或【V100】，点击保存
4.   点击右上角【连接】按钮，等待成功连接后会出现绿色对勾
5.   点击下方【安装环境并运行】运行按钮，等待10分钟左右，出现网址链接，任意点开一个即可进入
6.   之后就和本地部署一样了

后续使用只需云盘打开保存的副本colab，执行3、4、5即可

若页面连接失败或运行被终端，只需运行2、开始运行即可。





## 部署涵盖的插件列表：
1.   deforum
2.   controlnet
3.   openpose-editor
4.   depth-lib
5.   huggingface
6.   additional-networks
7.   posex
8.   3d-open-pose-editor
9.   webui-tunnels
10.   rembg
11.   two-shot
12.   aspect-ratio-helpe
13.   adetailer
14.   dynamic-thresholding
15.   dynamic-prompts
16.   wildcards
17.   segment-anything
18.   ch-ZN汉化
19.   multidiffusion-upscaler



# 安装与运行

In [ ]:
#@title 1、安装环境并运行
%cd /content
import urllib, requests
from notebook import notebookapp
name = requests.get(f"{list(notebookapp.list_running_servers())[0]['url']}api/sessions").json()[0]["name"].replace("_run", "")
url = urllib.parse.unquote("https%3A%2F%2Fraw.githubusercontent.com%2FChunLiangSian%2FSD_Ui%2Fmain%2FUI_15%2FCLSA_UI_Drive_15.ipynb")
!wget {url} -O {name}
%run {name}


In [ ]:
#@title 开始运行
a="stable"
b="Stable"
c="sd"

%cd /content
from google.colab import drive
drive.mount('/content/drive')
import os

%env PYTHONDONTWRITEBYTECODE=1
%env TF_CPP_MIN_LOG_LEVEL=1

!apt -y update -qq
!wget https://github.com/camenduru/gperftools/releases/download/v1.0/libtcmalloc_minimal.so.4 -O /content/libtcmalloc_minimal.so.4
%env LD_PRELOAD=/content/libtcmalloc_minimal.so.4

!apt -y install -qq aria2 libcairo2-dev pkg-config python3-dev
!apt -y install -qq unionfs-fuse
!pip install -q torch==2.0.0+cu118 torchvision==0.15.1+cu118 torchaudio==2.0.1+cu118 torchtext==0.15.1 torchdata==0.6.0 --extra-index-url https://download.pytorch.org/whl/cu118 -U
!pip install -q xformers==0.0.19 triton==2.0.0 -U

!mkdir /content/fused-models
!mkdir /content/models
!mkdir /content/fused-lora
!mkdir /content/lora
!unionfs-fuse /content/drive/MyDrive/ui-colab/ui/models/{b}-diffusion=RW:/content/models=RW /content/fused-models
!unionfs-fuse /content/drive/MyDrive/ui-colab/ui/extensions/{c}-webui-additional-networks/models/lora=RW:/content/drive/MyDrive/ui-colab/ui/models/Lora=RW:/content/lora=RW /content/fused-lora

%cd /content/drive/MyDrive/ui-colab/ui

!git reset --hard
!git -C /content/drive/MyDrive/ui-colab/ui/repositories/{a}-diffusion-stability-ai reset --hard


!git -C /content/drive/MyDrive/ui-colab/ui/extensions/{c}-webui-tunnels reset --hard
!git -C /content/drive/MyDrive/ui-colab/ui/extensions/{c}-webui-tunnels pull


!sed -i -e '''/    prepare_environment()/a\    os.system\(f\"""sed -i -e ''\"s/dict()))/dict())).cuda()/g\"'' /content/drive/MyDrive/ui-colab/ui/repositories/stable-diffusion-stability-ai/ldm/util.py""")''' /content/drive/MyDrive/ui-colab/ui/launch.py
!sed -i -e 's/checkout {commithash}/checkout --force {commithash}/g' /content/drive/MyDrive/ui-colab/ui/launch.py

if not os.path.exists(f"/content/drive/MyDrive/ui-colab/ui/models/CLIP"):
  os.mkdir(f"/content/drive/MyDrive/ui-colab/ui/models/CLIP")
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://openaipublic.azureedge.net/clip/models/b8cca3fd41ae0c99ba7e8951adf17d267cdb84cd88be6f7c2e0eca1737a03836/ViT-L-14.pt -d /content/drive/MyDrive/ui-colab/ui/models/CLIP -o ViT-L-14.pt

!python launch.py --listen --xformers --enable-insecure-extension-access --theme dark --gradio-queue --clip-models-path /content/drive/MyDrive/ui-colab/ui/models/CLIP --ckpt-dir /content/fused-models --lora-dir /content/fused-lora --multiple

In [ ]:
#@title 仅运行


%cd /content/drive/MyDrive/ui-colab/ui
!python launch.py --listen --xformers --enable-insecure-extension-access --theme dark --gradio-queue --clip-models-path /content/drive/MyDrive/ui-colab/ui/models/CLIP --ckpt-dir /content/fused-models --lora-dir /content/fused-lora --multiple

In [ ]:

#@title 查看GPU和VRAM可用信息

!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader

# 模型下载

In [ ]:

#@title Checkpoints/safetensors等大模型下载
url = "https://civitai.com/api/download/models/128713" #@param {type:"string"}
#@ markdown - _Remote source url._
destination_folder = urllib.parse.unquote("%2Fcontent%2Fdrive%2FMyDrive%2Fui-colab%2Fui%2Fmodels%2FStable-diffusion") #@ param {type:"string"}
#@ markdown - _Find folder in gdrive on the left and right-click to copy the path._
filename = "DreamShaper.safetensors" #@param {type:"string"}
#@ markdown - _Filename with extension._

target = f'{destination_folder}/{filename}'

!gdown -O $target "$url"


In [ ]:
#@title Lora模型下载
url = "https://civitai.com/api/download/models/6433" #@param {type:"string"}
#@ markdown - _Remote source url._
destination_folder = urllib.parse.unquote("%2Fcontent%2Fdrive%2FMyDrive%2Fui-colab%2Fui%2Fmodels%2FLora") #@ param {type:"string"}
#@ markdown - _Find folder in gdrive on the left and right-click to copy the path._
filename = "Eye_LoRa.safetensors" #@param {type:"string"}
#@ markdown - _Filename with extension._

target = f'{destination_folder}/{filename}'

!gdown --fuzzy -O $target "$url"

In [ ]:
#@title embeddings模型下载
url = "https://huggingface.co/datasets/g{c}f/EasyNegative/resolve/main/EasyNegative.safetensors" #@param {type:"string"}
#@ markdown - _Remote source url._
destination_folder = urllib.parse.unquote("%2Fcontent%2Fdrive%2FMyDrive%2Fui-colab%2Fui%2Fembeddings") #@ param {type:"string"}
#@ markdown - _Find folder in gdrive on the left and right-click to copy the path._
filename = "EasyNegative.safetensors" #@param {type:"string"}
#@ markdown - _Filename with extension._

target = f'{destination_folder}/{filename}'

!gdown --fuzzy -O $target "$url"

In [ ]:
#@title VAE下载
url = "https://huggingface.co/stabilityai/{c}-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors" #@param {type:"string"}
#@ markdown - _Remote source url._
destination_folder = urllib.parse.unquote("%2Fcontent%2Fdrive%2FMyDrive%2Fui-colab%2Fui%2Fmodels%2FVAE") #@ param {type:"string"}
#@ markdown - _Find folder in gdrive on the left and right-click to copy the path._
filename = "vae-ft-mse-840000-ema-pruned.safetensors" #@param {type:"string"}
#@ markdown - _Filename with extension._

target = f'{destination_folder}/{filename}'

!gdown --fuzzy -O $target "$url"